# Behaviors for Mobile Robot Control


### Differential-drive robot

A differential-drive has 2 driving wheels, one on its left and another on its right side. A third wheel (not motorized) is usually used to balance the structure. The difference in speed between the left and right wheels will change the orientation of the robot. 

You can create controllers that generate desired values for the speeds of left and right wheels. But, in this case, your controllers will only work for differential-drive robots. A more general approach is to create controllers that generate reference (desired) values of linear and angular speeds, and implement a function to convert those values into desired wheel speeds. An example of such function is shown below.

In [1]:
def wheel_speed_commands(u_d, w_d, D, R):
    """Converts desired speeds to wheel speed commands for a differential-drive robot.
    Inputs:
        u_d = desired linear speed for the robot [m/s]
        w_d = desired angular speed for the robot [rad/s]
        R = radius of the robot wheel [m]
        D = distance between the left and right wheels [m]
    Returns:
        wr_d = desired speed for the right wheel [rad/s]
        wl_d = desired speed for the left wheel [rad/s]
    """
    wr_d = float((2*u_d + D*w_d)/(2*R))
    wl_d = float((2*u_d - D*w_d)/(2*R))

    return wl_d, wr_d

Test the function with different desired values:

In [2]:
# Physical parameters of the robot for the kinematics model
R = 0.0205    # radius of the wheels of the e-puck robot: 20.5mm 
D = 0.0520    # distance between the wheels of the e-puck robot: 52mm

# Desired speeds:
u_d = 0.1   # [m/s]
w_d = -0.5  # [rad/s]

wl_d, wr_d = wheel_speed_commands(u_d, w_d, D, R)

print(f"Desired speed of the left wheel  = {wl_d} rad/s")
print(f"Desired speed of the right wheel = {wr_d} rad/s")


Desired speed of the left wheel  = 5.512195121951219 rad/s
Desired speed of the right wheel = 4.2439024390243905 rad/s


The above function was designed to convert desired linear and angular speeds into wheel speeds for a differential-drive robot. The purpose of such function is to serve as an abstraction layer between the robot moving controllers and the speed controllers of its own wheels. If you use a different structure (with omniwheels, for example), all you need to do is to change the equations to generate the speeds of each motor that corresponds to the desired linear and angular speeds. 

### Implementation of Robot Behaviors

#### Wall-following Behavior
This function implements a wall-following behavior with constant linear speed.


In [3]:
def follow_wall_to_left(kp, d_l, d_desired):
    """ Follows the wall to the left of the robot.
    Input Parameters: 
        kp = proportional controller gain;
        d_desired = desired robot distance to the wall;
        d_l = measured distance to the left wall;
    Returns:
        u_ref = reference linear speed command;
        w_ref = reference angluar speed command.
    """
    u_ref = 0.5     # [m/s] some constant linear speed
    w_ref = kp*(d_l - d_desired)
    
    return u_ref, w_ref

Assume you want to follow the wall at a distance of 20cm, and the robot is actually at a distance of 18cm. When the function is called, it will return the desired speed values that the robot should have to get closer to the wall.


In [4]:
d_desired = 0.20    # [m]
d_l = 0.18          # [m] - this value must be calculated based on sensor readings

# Proportional controller gain: defines how the reaction of the robot will be:
# higher controller gain will result in faster reaction, but it can cause oscillations
kp = 1

u_ref, w_ref = follow_wall_to_left(kp, d_l, d_desired)

print(f"Desired linear speed  = {u_ref}m/s")
print(f"Desired angular speed = {w_ref}rad/s")

Desired linear speed  = 0.5m/s
Desired angular speed = -0.020000000000000018rad/s


Now we can calculate the desired speed of each wheel using the previous function:


In [5]:
wl_d, wr_d = wheel_speed_commands(u_ref, w_ref, D, R)

print(f"Desired speed of the left wheel  = {wl_d} rad/s")
print(f"Desired speed of the right wheel = {wr_d} rad/s")

Desired speed of the left wheel  = 24.41560975609756 rad/s
Desired speed of the right wheel = 24.364878048780486 rad/s


#### Wall-following Behavior with Obstacle Detection
This function implements a wall-following behavior with linear speed that depends on the distance to obstacle.


In [6]:
def follow_wall_to_left_obst(kp, d_l, d_desired, d):
    """ Follows the wall to the left of the robot.
    Input Parameters: 
        kp = proportional controller gain;
        d_desired = desired robot distance to the wall;
        d_l = measured distance to the left wall;
        d = measured distance to the obstacle (front sensor);
    Returns:
        u_ref = reference linear speed command;
        w_ref = reference angluar speed command.
    """
    # Variables:
    d_min = 0.05     # [m] minimum admissible distance to the obstacle; 
    d_max = 0.50     # [m] maximum measurable distance by the front sensor;
    u_max = 0.5      # [m/s] linear speed for d = d_max

    u_ref = u_max*d/d_max if d >= d_min else 0
    w_ref = kp*(d_l - d_desired)
    
    return u_ref, w_ref

Assume you want to follow the wall at a distance of 20cm, and the robot is actually at a distance of 18cm. When the function is called, it will return the desired speed values that the robot should have to get closer to the wall.

Test the function with different values of `d`.


In [7]:
d = 0.5
d_desired = 0.20    # [m]
d_l = 0.18          # [m] - this value must be calculated based on sensor readings

# Proportional controller gain: defines how the reaction of the robot will be:
# higher controller gain will result in  faster reaction, but it can cause oscillations
kp = 1

u_ref, w_ref = follow_wall_to_left_obst(kp, d_l, d_desired, d)
wl_d, wr_d = wheel_speed_commands(u_ref, w_ref, D, R)

print(f"Desired linear speed  = {u_ref}m/s")
print(f"Desired angular speed = {w_ref}rad/s")
print(f"Desired speed of the left wheel  = {wl_d} rad/s")
print(f"Desired speed of the right wheel = {wr_d} rad/s")

Desired linear speed  = 0.5m/s
Desired angular speed = -0.020000000000000018rad/s
Desired speed of the left wheel  = 24.41560975609756 rad/s
Desired speed of the right wheel = 24.364878048780486 rad/s


The above implementations of wall-following behavior will likely cause the robot to perform poorly. To make sure that the robot follows the wall properly, it is a good idea to include another term that tries to maintain the robot parallel to it. For that, we assume that the robot has two sensors that measure its distance to the left wall, one in the front and another in the back of the robot. The sensors are placed on a distance _s_ from each other.

In [8]:
def improved_follow_wall_to_left_obst(kp, kp2, d_fl, d_rl, d_desired, d):
    """ Follows the wall to the left of the robot.
    Input Parameters: 
        kp = controller gain for controlling the distance to the wall;
        kp2 = controller gain for keeping the robot parallel to the wall;
        d_desired = desired robot distance to the wall;
        d_fl = distance to the left wall measured by the front sensor;
        d_rl = distance to the left wall measured by the rear sensor;
        d = measured distance to the obstacle (front sensor);
    Returns:
        u_ref = reference linear speed command;
        w_ref = reference angluar speed command.
    """
    # Variables:
    d_min = 0.05     # [m] minimum admissible distance to the obstacle; 
    d_max = 0.50     # [m] maximum measurable distance by the front sensor;
    u_max = 0.5      # [m/s] linear speed for d = d_max

    u_ref = u_max * d/d_max if d >= d_min else 0
    d_l = (d_fl + d_rl)/2
    w_ref = kp*(d_l - d_desired) + kp2*(d_fl - d_rl)
    
    return u_ref, w_ref

Test it with different values for _d_fl_ and _d_rl_.

In [9]:
d = 0.5
d_desired = 0.20    # [m]
d_fl = 0.18         # [m]
d_rl = 0.20         # [m]


# Controller gains: define how the reaction of the robot will be:
# higher controller gains will result in faster reaction, but can cause oscillations
kp = 1
kp2 = 1

u_ref, w_ref = improved_follow_wall_to_left_obst(kp, kp2, d_fl, d_rl, d_desired, d)
wl_d, wr_d = wheel_speed_commands(u_ref, w_ref, D, R)

print(f"Desired linear speed  = {u_ref}m/s")
print(f"Desired angular speed = {w_ref}rad/s")
print(f"Desired speed of the left wheel  = {wl_d} rad/s")
print(f"Desired speed of the right wheel = {wr_d} rad/s")

Desired linear speed  = 0.5m/s
Desired angular speed = -0.030000000000000027rad/s
Desired speed of the left wheel  = 24.428292682926827 rad/s
Desired speed of the right wheel = 24.35219512195122 rad/s


### Conclusion

After completing this notebook, you should understand how to build functions to implement simple behaviors for generic mobile robots. You are also able to convert the desired linear and angular speeds generated by the behaviors into speed commands for the wheels of a differential-drive robot. 